Implementation of the co-training algorithm proposed by Blum and Mitchell (1998).


In [0]:
import os
import string
import re
import numpy as np
from bs4 import BeautifulSoup
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

The data consists of HTML scraped from the webpages. As we're interested in the text contents only, we will clean this up by removing the HTML tags, converting all the words to lower case (as proper case and lower case words will be treated the same), and remove 'stop words' like 'the' which are commonly occuring words but probably do not contribute much to the classifier. We will also replace numeric values with 'num' as these are likely to relate to different courses. Much of the help came from this [Cambridge Spark](https://https://blog.cambridgespark.com/tutorial-preprocessing-text-data-a8969189b779) tutorial.

The original paper uses a bag of words representation, so we will do the same.

In [0]:
def load_data(directory):
  text_array = []
  # Load all of the files for each view
  for path, dirs, files in os.walk(directory):
    for filename in files:
      fname = os.path.join(path, filename)
      with open(fname, 'rb') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        # Remove the HTML tags and convert all to lowercase as case does not matter
        text_array.append(soup.get_text().lower())
  text_array = [re.sub(r'\d+', 'num', page) for page in text_array]
  return text_array

In [0]:
def process_data(text_array):
  counter = CountVectorizer(stop_words='english')
  bag_of_words = counter.fit_transform(text_array).toarray()
  return counter, bag_of_words

We will create accompanying labels, setting course to '1' and non-course to '0' .

In [0]:
num_course = np.ones((230,1))
num_noncourse = np.zeros((821,1))
labels = np.vstack((num_course, num_noncourse))

In [0]:
# Load data from both views
fulltext_dir ='course-cotrain-data/fulltext/'
inlinks_dir = 'course-cotrain-data/inlinks/'

fulltext_data = load_data(fulltext_dir)
inlinks_data = load_data(inlinks_dir)

# Create BoW representations for both views
f_counter, f_bow = process_data(fulltext_data)
i_counter, i_bow = process_data(inlinks_data)

# Comparison: train a naive Bayes classifier on a single view

For comparison, we will compare the co-training method to naive Bayes classifiers trained on a single view only. Note that the below setting is supervised and not semi-supervised.

As the dataset is rather imbalanced as there are more non-course samples than course samples, accuracy would not be a reliable metric. Therefore we will use the F1-score which is a weighted average of precision ($\frac{\textrm{true positive}}{\textrm{true positive}+\textrm{false positive}}$) and recall ($\frac{\textrm{true positive}}{\textrm{true positive}+\textrm{false negative}}$).

$$F_1 = \frac{2\times\textrm{precision}\times\textrm{recall}}{\textrm{precision}+\textrm{recall}}$$

In [0]:
# For comparision, we will make the same split for each classifier
f_train, f_test, i_train, i_test, y_train, y_test = train_test_split(f_bow, i_bow, labels.ravel(), test_size=0.25)

## Fulltext data only

In [9]:
# Initialise a naive Bayes classifier
f_gnb = MultinomialNB()

# Train and predict on the test set
f_pred = f_gnb.fit(f_train, y_train).predict(f_test)

# Print the confusion matrix and F1 score
print("Confusion matrix: \n {} \n F1 Score: {}".format(confusion_matrix(y_test, f_pred),f1_score(y_test, f_pred)))

Confusion matrix: 
 [[166  30]
 [ 48  19]] 
 F1 Score: 0.3275862068965517


## Inlinks data only

In [10]:
i_gnb = MultinomialNB()

i_pred = i_gnb.fit(i_train, y_train).predict(i_test)

print("Confusion matrix: \n {} \n F1 Score: {}".format(confusion_matrix(y_test, i_pred),f1_score(y_test, i_pred)))

Confusion matrix: 
 [[100  96]
 [ 32  35]] 
 F1 Score: 0.35353535353535354


# Co-Training

The co-training paper uses 25% of the data as the test data and the remaining as the training data. Within the training data, we generate three subsets: $L$ (the labelled data), $U$ (the unlabelled data), and $U'$ (a subset of $U$). 

Initially $L$ comprises of 3 positive (course) and 9 negative (non-course) samples. This is generated below.

In [0]:
L_i_positive = i_train[np.where(y_train ==1)[0][:3],:]
L_i_negative = i_train[np.where(y_train != 1)[0][:9],:]
L_i = np.vstack((L_i_positive, L_i_negative))

# This is the complement of L_i
U_i = np.vstack((i_train[np.where(y_train ==1)[0][3:],:], i_train[np.where(y_train != 1)[0][9:],:]))

L_f_positive = f_train[np.where(y_train ==1)[0][:3],:]
L_f_negative = f_train[np.where(y_train != 1)[0][:9],:]
L_f = np.vstack((L_f_positive, L_f_negative))

# This is the complement of L_f
U_f = np.vstack((f_train[np.where(y_train ==1)[0][3:],:], f_train[np.where(y_train != 1)[0][9:],:]))

# Create the labels
L_labels = np.vstack((np.ones((3,1)), np.zeros((9,1)))).ravel()

We now implement the co-training algorithm. As we are continually adding samples to $L$, training the Naive Bayes classifier directly consumes increasingly more memory. Therefore we have to compromise and train in randomly sampled batches using a partial fit, meaning that the number of iterations for which the classifier is trained is different to what is specified in the paper.

In [0]:
# Function to generate U' from random samples of U
def sample_U(U_i, U_f, u = 75):
  # We use the random indices to pick random samples
  random_indices = np.arange(0, U_i.shape[0])
  np.random.shuffle(random_indices)
  U_i_prime = U_i[random_indices[:u],:]
  U_f_prime = U_f[random_indices[:u],:]

  # U is now the complement of U'
  U_i = U_i[random_indices[:u],:]
  U_f = U_f[random_indices[:u],:]

  return U_i_prime, U_f_prime, U_i, U_f

# Function which updates L by adding the most confident positive and negative values from each classifier
def label_from_U(U_f_prime, U_i_prime, f_scores, i_scores, L_f, L_i, L_labels, p = 1, n = 3):

  f_scores_pos = np.argsort(f_scores[:,1], axis=0)[::-1][:p]
  f_scores_neg = np.argsort(f_scores[:,0], axis=0)[::-1][:n]
  i_scores_pos = np.argsort(i_scores[:,1], axis=0)[::-1][:p]
  i_scores_neg = np.argsort(i_scores[:,0], axis=0)[::-1][:n]

  # Get the complement of the above scores so that U' can be updated. Needs to include predictions from both classifiers
  scores_pos = np.union1d(np.argsort(f_scores[:,1], axis=0)[::-1][p:], np.argsort(i_scores[:,1], axis=0)[::-1][p:])
  scores_neg = np.union1d(np.argsort(f_scores[:,0], axis=0)[::-1][n:], np.argsort(i_scores[:,0], axis=0)[::-1][n:])

  f_pos = U_f_prime[f_scores_pos,:]
  f_neg = U_f_prime[f_scores_neg,:]
  fi_pos = U_i_prime[f_scores_pos,:]
  fi_neg = U_i_prime[f_scores_neg,:]
  i_pos = U_i_prime[i_scores_pos,:]
  i_neg = U_i_prime[i_scores_neg,:]
  if_pos = U_f_prime[i_scores_pos,:]
  if_neg = U_f_prime[i_scores_neg,:]

  # Add most confident values from both classifiers to L
  L_f = np.vstack((L_f, f_pos, if_pos, f_neg, if_neg))
  L_i = np.vstack((L_i, i_pos, fi_pos, i_neg, fi_neg))

  # Update labels
  L_labels = np.vstack((L_labels[:,np.newaxis], np.ones((2*p,1)), np.zeros((2*n,1)))).ravel()

  return L_f, L_i, L_labels, scores_pos, scores_neg

# Function to sample from L
def batch_sample(L_f, L_i, L_labels, sample_size = 16):
  random_indices = np.arange(0, L_f.shape[0])
  np.random.shuffle(random_indices)
  batch_f = L_f[random_indices[:sample_size],:]
  batch_i = L_i[random_indices[:sample_size],:]
  batch_labels = L_labels[random_indices[:sample_size]]

  return batch_f, batch_i, batch_labels

def cotrain(L_i, U_i, L_f, U_f, L_labels, f_test, i_test, y_test, p = 1, n = 3, k = 30, u = 75):
  # Generate U'
  U_i_prime, U_f_prime, U_i, U_f = sample_U(U_i, U_f, u = u)

  # Train classifiers on the original L
  # Train h1
  f_gnb = MultinomialNB()
  # Get the confidence scores for h1
  f_scores = f_gnb.fit(L_f, L_labels).predict_proba(U_f_prime)
  # Train h2
  i_gnb = MultinomialNB()
  # Get  the confidence scores for h2
  i_scores = i_gnb.fit(L_i, L_labels).predict_proba(U_i_prime)

  L_f, L_i, L_labels, _, _ = label_from_U(U_f_prime, U_i_prime, f_scores, i_scores, L_f, L_i, L_labels)

  # Loop for k iterations
  for count in range(k - 1):
    batch_f, batch_i, batch_labels = batch_sample(L_f, L_i, L_labels)
    # Partially fit h1 and h2 using the batch
    f_scores = f_gnb.partial_fit(batch_f, batch_labels).predict_proba(U_f_prime)
    i_scores = i_gnb.partial_fit(batch_i, batch_labels).predict_proba(U_i_prime)
    # At the final iteration, record the final predictions which will be used to measure performance
    if count == k - 2:
      f_pred = f_gnb.predict(f_test)
      i_pred = i_gnb.predict(i_test)
    L_f, L_i, L_labels, scores_pos, scores_neg = label_from_U(U_f_prime, U_i_prime, f_scores, i_scores, L_f, L_i, L_labels)

    # Remove labelled elements from U'
    U_f_prime = np.vstack((U_f_prime[scores_pos,:], U_f_prime[scores_neg,:]))
    U_i_prime = np.vstack((U_i_prime[scores_pos,:], U_i_prime[scores_neg,:]))

    # Replenish U'
    num_new_samples = 2*p + 2*n
    U_i_prime, U_f_prime, U_i, U_f = sample_U(U_i, U_f, u = num_new_samples)

  return f_pred, i_pred

In [0]:
f_pred, i_pred = cotrain(L_i, U_i, L_f, U_f, L_labels, f_test, i_test, y_test, k =300)

The results are more variable as we are using batches but there is a slight improvement in the F1 score for both classifiers.

In [14]:
# Print the confusion matrix and F1 score for the co-trained classifer trained on fulltext
print("Confusion matrix: \n {} \n F1 Score: {}".format(confusion_matrix(y_test, f_pred),f1_score(y_test, f_pred)))

Confusion matrix: 
 [[123  73]
 [ 35  32]] 
 F1 Score: 0.37209302325581395


In [15]:
# Print the confusion matrix and F1 score for the co-trained classifer trained on inlinks
print("Confusion matrix: \n {} \n F1 Score: {}".format(confusion_matrix(y_test, i_pred),f1_score(y_test, i_pred)))

Confusion matrix: 
 [[ 47 149]
 [  8  59]] 
 F1 Score: 0.4290909090909091
